In [ ]:
import os

from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder \
    .appName("ReadFromRDS") \
    .config("spark.jars", "/home/ollyhill/data-testing/mysql-connector-j-8.4.0.jar") \
    .getOrCreate()


# JDBC connection properties
jdbc_url = (
    "jdbc:mysql://" +
    os.environ["PORTAL_RDS"] +
    ":3306/" +
    os.environ["PORTAL_DB"]
)
connection_properties = {
    "user": os.environ['PORTAL_USER'],
    "password": os.environ['PORTAL_PASSWORD'],
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Load table into DataFrame
df = spark.read.jdbc(
    url=jdbc_url,
    table="api_calls",
    properties=connection_properties
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/23 15:32:40 WARN Utils: Your hostname, OH-LAPTOP, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/23 15:32:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/23 15:32:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/23 15:32:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


25/07/24 08:20:20 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 37566969 ms exceeds timeout 120000 ms
25/07/24 08:20:20 WARN SparkContext: Killing executors is not supported by current scheduler.
25/07/24 08:20:25 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint

In [ ]:
from pyspark.sql import functions as F

df_with_date = df.withColumn("created_date", F.to_date("created_at"))

unique_combinations = df_with_date \
    .filter(
        (F.col("user_id").isNotNull()) &
        (F.col("device_id").isNotNull()) &
        (F.col("created_date").isNotNull()) &
        (F.col("created_date") >= F.lit("2025-06-01"))
    ) \
    .select("user_id", "device_id", "created_date") \
    .distinct()


In [4]:
unique_combinations.show()

+-------+---------+------------+
|user_id|device_id|created_date|
+-------+---------+------------+
|  11581|    12436|  2025-06-01|
|  11637|    14395|  2025-06-01|
|  18575|    24956|  2025-06-01|
|   3020|    24567|  2025-06-02|
|   4358|     6966|  2025-06-02|
|  18184|    25083|  2025-06-02|
|  13588|    16592|  2025-06-03|
|   3581|    10183|  2025-06-03|
|  16080|    21465|  2025-06-03|
|   2318|    16999|  2025-06-03|
|   7671|    25517|  2025-06-03|
|  13781|    17982|  2025-06-04|
|  18456|    24774|  2025-06-04|
|  17122|    25673|  2025-06-04|
|   3471|    22499|  2025-06-04|
|  18488|    24434|  2025-06-05|
|   9487|    20277|  2025-06-05|
|   7938|    24228|  2025-06-05|
|   4607|    18575|  2025-06-06|
|  12897|    21702|  2025-06-06|
+-------+---------+------------+
only showing top 20 rows
